In [1]:
import math
import pandas as pd
import numpy as np
np.random.seed(42)
from collections import OrderedDict

%matplotlib notebook
%matplotlib notebook
import matplotlib.pyplot as plt

import matplotlib
matplotlib.rcParams['font.size'] = 16

matplotlib.rcParams['figure.figsize'] = (30,9)

import seaborn as sns

from IPython.core.pylabtools import figsize

from scipy.stats import percentileofscore
from scipy import stats

In [2]:
df = pd.read_csv('dataset/features/part-00000-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df.head()

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed
0,1202590843006,3.000,353.0,1.228867,8.900100,3.986968,0.008221,0.002269,-0.009966,1362.0,0.000000
1,274877907034,9.293,17.0,0.032775,8.659933,4.737300,0.024629,0.004028,-0.010858,257.0,0.190000
2,884763263056,3.000,189.0,1.139675,9.545974,1.951334,-0.006899,-0.015080,0.001122,973.0,0.667059
3,1073741824054,3.900,126.0,3.871543,10.386364,-0.136474,0.001344,-0.339601,-0.017956,902.0,7.913285
4,1056561954943,3.900,50.0,-0.112882,10.550960,-1.560110,0.130568,-0.061697,0.161530,820.0,20.419409


In [3]:
df.describe()

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed
count,1.613554e+06,1.613554e+06,1.613554e+06,1.613554e+06,1.613554e+06,1.613554e+06,1.613554e+06,1.613554e+06,1.613554e+06,1.613554e+06,1.613554e+06
mean,8.185688e+11,1.157350e+01,1.689111e+02,6.807467e-02,4.469578e+00,8.985294e-01,-1.754218e-03,4.358774e-04,-2.671857e-04,4.824088e+03,9.009763e+00
std,4.951339e+11,8.665737e+01,1.072641e+02,1.426141e+00,8.129320e+00,3.249632e+00,1.430876e-01,3.460380e-01,1.525665e-01,1.617019e+06,8.106659e+00
min,0.000000e+00,7.500000e-01,0.000000e+00,-3.344084e+01,-5.730359e+01,-7.844842e+01,-2.644802e+01,-5.734489e+01,-5.355445e+01,0.000000e+00,-1.719776e+00
25%,3.779571e+11,3.900000e+00,7.800000e+01,-5.087678e-01,-1.995674e+00,-9.265844e-01,-2.686586e-02,-2.983093e-02,-1.867207e-02,2.420000e+02,1.019846e+00
50%,8.074539e+11,4.259000e+00,1.682945e+02,6.130981e-02,9.082103e+00,7.807281e-01,-6.597335e-04,2.746582e-04,-3.051758e-05,5.200000e+02,7.540000e+00
75%,1.254130e+12,8.000000e+00,2.624448e+02,6.344828e-01,9.710000e+00,2.753416e+00,2.333505e-02,3.141879e-02,1.825624e-02,8.630000e+02,1.548176e+01
max,1.709397e+12,6.063000e+03,3.599985e+02,2.961647e+01,7.281000e+01,7.331421e+01,1.068810e+01,6.254202e+01,3.383183e+01,6.193165e+08,5.055000e+01


In [4]:
def count_nulls(df):
    null_counter = df.isnull().sum(axis=0)
    null_counter = null_counter[null_counter > 0]
    null_percent = df.isnull().sum(axis=0) / df.shape[0] * 100
    null_percent = null_percent[null_percent > 0]
    null_df = pd.concat([null_counter,null_percent],axis=1)
    null_df.columns = ['count','percent']
    display(null_df)

In [7]:
count_nulls(df)

,count,percent


In [10]:
# str_target_var="bookingId"
# ls_num_data = [col for col in df.columns if df[col].dtype != "object" and col != str_target_var]
# ncols = 3
# nrows = int(math.ceil(len(ls_num_data) * 1.0 / ncols))
# fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 30))

# # Visualization of numerical data
# for idx, var in enumerate(ls_num_data):
#     i = idx / ncols
#     j = idx % ncols
#     axes[i, j].get_xaxis().get_label().set_visible(False)
#     df.boxplot(column=var, by=str_target_var, ax=axes[i, j])
df.dtypes

bookingID           int64
Accuracy          float64
Bearing           float64
acceleration_x    float64
acceleration_y    float64
acceleration_z    float64
gyro_x            float64
gyro_y            float64
gyro_z            float64
second            float64
Speed             float64
dtype: object

In [ ]:
ax = df['Speed'].plot.kde(bw_method=0.3)

In [21]:
df[df["bookingID"]==1202590843006]

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed
0,1.202591e+12,3.000,353.0,1.228867,8.900100,3.986968,0.008221,0.002269,-0.009966,1362.0,0.000000
36004,1.202591e+12,3.033,77.0,1.137217,8.975278,3.981050,0.021118,-0.022515,0.024522,541.0,0.000000
37792,1.202591e+12,6.173,283.0,1.522818,7.947064,5.075056,0.137567,-0.297055,-0.017418,154.0,2.863286
57021,1.202591e+12,3.014,353.0,1.134775,8.932489,3.929421,-0.007959,-0.020543,-0.022986,1384.0,0.000000
76231,1.202591e+12,5.209,340.0,0.858197,9.078037,2.944571,0.061924,0.015219,0.025517,1142.0,9.750134
93956,1.202591e+12,3.014,326.0,1.030723,9.093197,3.593553,0.019164,0.004416,0.001449,812.0,0.000000
101646,1.202591e+12,3.000,315.0,0.938498,8.735906,4.567591,0.007278,-0.017104,-0.025045,1046.0,0.000000
122477,1.202591e+12,16.027,135.0,1.618471,7.706810,6.337251,-0.010140,-0.069377,0.014224,46.0,12.496430
130810,1.202591e+12,3.021,75.0,0.459601,8.632447,2.828108,0.014172,0.088750,0.031381,544.0,0.000000
188511,1.202591e+12,13.021,343.0,1.998440,9.150648,3.737292,-0.000262,-0.072780,-0.060266,579.0,0.725661


In [22]:
df1 = pd.read_csv('dataset/features/part-00001-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df2 = pd.read_csv('dataset/features/part-00002-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df3 = pd.read_csv('dataset/features/part-00003-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df4 = pd.read_csv('dataset/features/part-00004-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df5 = pd.read_csv('dataset/features/part-00005-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df6 = pd.read_csv('dataset/features/part-00006-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df7 = pd.read_csv('dataset/features/part-00007-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df8 = pd.read_csv('dataset/features/part-00008-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df9 = pd.read_csv('dataset/features/part-00009-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')


In [31]:
print(df1[df1["bookingID"]==1202590843006].shape)
print(df2[df2["bookingID"]==1202590843006].shape)
print(df3[df3["bookingID"]==1202590843006].shape)
print(df4[df4["bookingID"]==1202590843006].shape)
print(df5[df5["bookingID"]==1202590843006].shape)
print(df6[df6["bookingID"]==1202590843006].shape)
print(df7[df7["bookingID"]==1202590843006].shape)
print(df8[df8["bookingID"]==1202590843006].shape)
print(df9[df9["bookingID"]==1202590843006].shape)

(120, 11)
(103, 11)
(115, 11)
(110, 11)
(119, 11)
(96, 11)
(119, 11)
(123, 11)
(127, 11)


(103, 11)